STEP 1: Import Libraries

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
import warnings

warnings.filterwarnings("ignore", category=FutureWarning, module='sklearn')

STEP 2: Load and Preview Data

In [ ]:
# Load the dataset
data = pd.read_csv('/kaggle/input/data-science-jobs-and-salary-glassdoor/glassdoor_jobs.csv')

# Display the first few rows of the dataframe
print(data.head())


STEP 3: Drop Unnecessary Column

In [ ]:
# Drop the 'Unnamed: 0' column
data.drop(columns=['Unnamed: 0'], inplace=True)


Cell 4: Check for Missing Values

In [ ]:
# Check for missing values
print(data.isnull().sum())


Cell 5: Parse Salary Estimates

In [ ]:
# Function to parse salary estimates
def parse_salary(salary):
    try:
        salary = salary.replace('(Glassdoor est.)', '').replace('K', '').replace('$', '').split('-')
        return [int(s.strip()) for s in salary]
    except Exception as e:
        print(f"Error parsing salary: {salary}, error: {e}")
        return [np.nan, np.nan]

# Apply the function and create new columns
data['Min Salary'], data['Max Salary'] = zip(*data['Salary Estimate'].apply(parse_salary))
data['Avg Salary'] = (data['Min Salary'] + data['Max Salary']) / 2

# Drop the original 'Salary Estimate' column
data.drop(columns=['Salary Estimate'], inplace=True)


Cell 6: Encode Categorical Variables and Select Relevant Columns

In [ ]:
# Clean and encode categorical variables
data['Company Name'] = data['Company Name'].apply(lambda x: x.split('\n')[0])

# Select relevant columns for modeling
model_data = data[['Job Title', 'Avg Salary', 'Rating', 'Location', 'Size', 'Type of ownership', 'Industry', 'Sector', 'Revenue']]

# Encode categorical variables
model_data = pd.get_dummies(model_data, drop_first=True)

# Handle any remaining missing values by filling with the mean or a fixed value
model_data.fillna(model_data.mean(), inplace=True)

# Display the cleaned and processed data
print(model_data.head())


Cell 7: Standardize Numerical Features

In [ ]:
# Standardize the numerical features
scaler = StandardScaler()
numerical_features = ['Rating']
model_data[numerical_features] = scaler.fit_transform(model_data[numerical_features])


Cell 8: Split the Data

In [ ]:
# Split the data into training and testing sets
X = model_data.drop(columns=['Avg Salary'])
y = model_data['Avg Salary']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


Cell 9: Initialize and Train Models

In [ ]:
# Initialize models
lr = LinearRegression()
rf = RandomForestRegressor(n_estimators=100, random_state=42)
gbr = GradientBoostingRegressor(n_estimators=100, random_state=42)
svr = SVR()

# Train and evaluate Linear Regression
lr.fit(X_train, y_train)
y_pred_lr = lr.predict(X_test)
lr_mse = mean_squared_error(y_test, y_pred_lr)
lr_r2 = r2_score(y_test, y_pred_lr)


Cell 10: Train and Evaluate Random Forest

In [ ]:
# Train and evaluate Random Forest
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)
rf_mse = mean_squared_error(y_test, y_pred_rf)
rf_r2 = r2_score(y_test, y_pred_rf)


Cell 11: Train and Evaluate Gradient Boosting

In [ ]:
# Train and evaluate Gradient Boosting
gbr.fit(X_train, y_train)
y_pred_gbr = gbr.predict(X_test)
gbr_mse = mean_squared_error(y_test, y_pred_gbr)
gbr_r2 = r2_score(y_test, y_pred_gbr)


Cell 12: Train and Evaluate Support Vector Machine

In [ ]:
# Train and evaluate Support Vector Machine
svr.fit(X_train, y_train)
y_pred_svr = svr.predict(X_test)
svr_mse = mean_squared_error(y_test, y_pred_svr)
svr_r2 = r2_score(y_test, y_pred_svr)


Cell 13: Print Evaluation Metrics

In [ ]:
# Print the mean squared error and R² score for each model
print(f'Linear Regression MSE: {lr_mse}, R²: {lr_r2}')
print(f'Random Forest MSE: {rf_mse}, R²: {rf_r2}')
print(f'Gradient Boosting MSE: {gbr_mse}, R²: {gbr_r2}')
print(f'Support Vector Machine MSE: {svr_mse}, R²: {svr_r2}')

In [ ]:
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}

In [ ]:
warnings.filterwarnings("ignore", category=FutureWarning)

param_grid = {
    'n_estimators': [100, 200, 300],
    'max_features': [1, 'sqrt', 'log2'],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}
    
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=3, n_jobs=-1, verbose=0)

# Fit GridSearchCV
grid_search.fit(X_train, y_train)

# Get the best parameters and best estimator
best_params = grid_search.best_params_
best_rf = grid_search.best_estimator_

# Predict and evaluate the model
y_pred = best_rf.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Print the best parameters, mean squared error, and R² score
print(f'Best Parameters: {best_params}')
print(f'Mean Squared Error: {mse}')
print(f'R² Score: {r2}')

In [ ]:
data['Predicted Avg Salary'] = best_rf.predict(model_data.drop(columns=['Avg Salary']))

# Save the data with predictions
data_with_predictions = data.copy()
data_with_predictions.to_csv('data_with_predictions.csv', index=False)

# Print the first few rows of the new data with predictions
print(data_with_predictions.head())

In [ ]:
import joblib

#'best_rf' is your best RandomForest model obtained from GridSearchCV
joblib.dump(best_rf, 'best_rf_model.pkl')

Pandas AI

In [ ]:
!pip install pandasai

In [ ]:
"""
import pandasai
import os
import pandas as pd
from pandasai import Agent


# By default, unless you choose a different LLM, it will use BambooLLM.
# You can get your free API key signing up at https://pandabi.ai (you can also configure it in your .env file)
os.environ["PANDASAI_API_KEY"] = "$2a$10$h4x6w6qdSxF/8eAvcsCmbu68Yy3SD4Bv2iAi.bruNx3PdclzmDh8y"

agent = Agent(model_data)
agent.chat('Give me the top highest paid roles')
"""